In [1]:
import pathlib
from utils import get_filenames_of_path

In [ ]:
root = pathlib.Path("../Indiv_Proj-main")

inputs = get_filenames_of_path(root / 'Dataset')
inputs.sort()


for idx, path in enumerate(inputs):
    old_name = path.stem
    old_extension = path.suffix
    dir = path.parent
    new_name = str(idx).zfill(3) + old_extension
    path.rename(pathlib.Path(dir, new_name))

In [ ]:
from pytorch_faster_rcnn.visual import Annotator

dir = pathlib.Path("../Indiv_Proj-main")
image_files = get_filenames_of_path(dir / 'Dataset')

annotator = Annotator(image_ids=image_files)
annotator.napari()

In [ ]:
annotator.add_class(label='Non-HCC', color='yellow')

In [ ]:
annotator.export_all(pathlib.Path('../Indiv_Proj-main/target'))

In [7]:
import torch

root = pathlib.Path("../Indiv_Proj-main")

targets = get_filenames_of_path(root / 'target')
targets.sort()

annotation = torch.load(targets[0])

UnpicklingError: invalid load key, '{'.

In [3]:
len(targets)

75

In [ ]:
targets.sort()

In [4]:
targets[0]

WindowsPath('../Indiv_Proj-main/target/000.json')